In [87]:
from epw import epw

In [88]:
import pandas as pd
CRC_Data = pd.read_csv('CRC_REAL.csv')

In [89]:
new_column_names = ["Hour", "Temperature", "Dew Point", "Humidity", "Wind", 
                    "Speed", "Gust", "Pressure", "Precip.Rate.", 
                    "Precip.Accum.", "UV", "Solar"]
CRC_Data.columns = new_column_names + [CRC_Data.columns[-1]]

In [90]:
for column in CRC_Data.columns:
    if column not in ["Hour", "Wind", "Date"]:
        CRC_Data[column] = CRC_Data[column].astype(str).str.replace(r'[^\d.]', '', regex=True)

In [91]:
CRC_Data = CRC_Data.drop(index=[0, 1])
CRC_Data = CRC_Data.reset_index(drop=True)


In [92]:
# Convert 'Temperature' column from Fahrenheit to Celsius
CRC_Data['Temperature'] = pd.to_numeric(CRC_Data['Temperature'], errors='coerce')
CRC_Data['Temperature'] = (CRC_Data['Temperature'] - 32) * 5.0 / 9.0

# Convert 'Dew Point' column from Fahrenheit to Celsius
CRC_Data['Dew Point'] = pd.to_numeric(CRC_Data['Dew Point'], errors='coerce')
CRC_Data['Dew Point'] = (CRC_Data['Dew Point'] - 32) * 5.0 / 9.0

# Define wind direction to degrees mapping
wind_to_degrees = {
    'North': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'East': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'South': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'West': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5
}

# Convert 'Wind' column from directions to degrees
CRC_Data['Wind'] = CRC_Data['Wind'].map(wind_to_degrees)

# Convert 'Speed' column to numeric and handle non-numeric values
CRC_Data['Speed'] = pd.to_numeric(CRC_Data['Speed'], errors='coerce')

# Fill any NaN values in 'Speed' with 0 or drop them if necessary
CRC_Data['Speed'] = CRC_Data['Speed'].fillna(0)  # Use .dropna() if you prefer to remove rows

# Convert wind speed from m/s to km/h
conversion_factor = 0.44704
CRC_Data['Speed'] = CRC_Data['Speed'] * conversion_factor

# Convert 'Pressure', 'Precip.Rate.', and 'Precip.Accum.' from inches to meters
inch_to_meter = 0.0254
CRC_Data['Precip.Rate.'] = pd.to_numeric(CRC_Data['Precip.Rate.'], errors='coerce') * inch_to_meter
CRC_Data['Precip.Accum.'] = pd.to_numeric(CRC_Data['Precip.Accum.'], errors='coerce') * inch_to_meter

In [93]:
CRC_Data

,Hour,Temperature,Dew Point,Humidity,Wind,Speed,Gust,Pressure,Precip.Rate.,Precip.Accum.,UV,Solar,Date
0,12:00 AM,13.777778,12.777778,93,270.0,0.0,,29.91,0.0,0.0,0.0,0,2023-01-01
1,12:06 AM,13.777778,12.777778,93,225.0,0.0,,29.92,0.0,0.0,0.0,0,2023-01-01
2,12:12 AM,13.777778,12.777778,93,247.5,0.0,,29.91,0.0,0.0,0.0,0,2023-01-01
3,12:18 AM,13.722222,12.777778,93,247.5,0.0,,29.92,0.0,0.0,0.0,0,2023-01-01
4,12:24 AM,13.722222,12.777778,93,225.0,0.0,,29.92,0.0,0.0,0.0,0,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54348,11:30 PM,8.388889,1.111111,60,225.0,0.0,,30.01,0.0,0.0,0.0,0,2023-12-31
54349,11:36 PM,8.388889,1.111111,60,225.0,0.0,,30.01,0.0,0.0,0.0,0,2023-12-31
54350,11:42 PM,8.388889,1.111111,60,225.0,0.0,,30.00,0.0,0.0,0.0,0,2023-12-31
54351,11:48 PM,8.388889,1.111111,60,225.0,0.0,,30.00,0.0,0.0,0.0,0,2023-12-31


In [94]:
testing = CRC_Data.copy()

In [95]:
testing['Datetime'] = pd.to_datetime(testing['Date'] + ' ' + testing['Hour'], format='%Y-%m-%d %I:%M %p', errors='coerce')

testing['Year'] = testing['Datetime'].dt.year
testing['Month'] = testing['Datetime'].dt.month
testing['Day'] = testing['Datetime'].dt.day

testing['Hour'] = testing['Datetime'].dt.hour

testing['Hour'] = testing['Hour'] + 1

In [96]:
#set index as the date
testing.set_index('Datetime', inplace=True)

numeric_columns = ['Temperature', 'Dew Point', 'Wind', 'Speed', 'Pressure', 'Precip.Rate.', 'Precip.Accum.']
for col in numeric_columns:
    testing[col] = pd.to_numeric(testing[col], errors='coerce')
    
#these 4 kept disappearing
for col in ['Humidity', 'Gust', 'UV', 'Solar']:
    testing[col] = pd.to_numeric(testing[col], errors='coerce')

# Resample by the specified length and calculate the average for each period
resample_len = 1
hourly_data = testing.resample(f'{resample_len}H').mean()

full_date_range = pd.date_range(start=testing.index.min(), end=testing.index.max(), freq='H')
full_date_range_df = pd.DataFrame(index=full_date_range)

#left join
hourly_data = full_date_range_df.join(hourly_data, how='left')

# Fill NaNs with the mean of each column
hourly_data.fillna(hourly_data.mean(), inplace=True)

# Reset index to turn 'Datetime' back into a column
hourly_data.reset_index(inplace=True)
hourly_data.rename(columns={'index': 'Datetime'}, inplace=True)

# Adjust the hour to start from 1 instead of 0
hourly_data['Hour'] = hourly_data['Datetime'].dt.hour + 1

# Add 'Date' column based on 'Datetime'
hourly_data['Date'] = hourly_data['Datetime'].dt.date

# Reorder columns if necessary
columns_order = ['Datetime', 'Hour', 'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Speed', 'Gust', 'Pressure', 'Precip.Rate.', 'Precip.Accum.', 'UV', 'Solar', 'Date']
# Only include columns that are present in hourly_data
columns_order = [col for col in columns_order if col in hourly_data.columns]

# Reorder DataFrame
hourly_data = hourly_data[columns_order]

print(hourly_data.head())

             Datetime  Hour  Temperature  Dew Point  Humidity    Wind  Speed  \
0 2023-01-01 00:00:00     1    13.677778  12.777778      93.2  258.75    0.0   
1 2023-01-01 01:00:00     2    13.472222  12.777778      94.0  249.75    0.0   
2 2023-01-01 02:00:00     3    13.527778  12.777778      94.0  279.00    0.0   
3 2023-01-01 03:00:00     4    13.055556  12.111111      94.0  290.25    0.0   
4 2023-01-01 04:00:00     5    12.422222  11.666667      94.0  234.00    0.0   

   Gust  Pressure  Precip.Rate.  Precip.Accum.   UV  Solar        Date  
0   NaN    29.913           0.0            0.0  0.0    0.0  2023-01-01  
1   NaN    29.904           0.0            0.0  0.0    0.0  2023-01-01  
2   NaN    29.911           0.0            0.0  0.0    0.0  2023-01-01  
3   NaN    29.918           0.0            0.0  0.0    0.0  2023-01-01  
4   NaN    29.923           0.0            0.0  0.0    0.0  2023-01-01  


C:\Users\zeyuj\AppData\Local\Temp\ipykernel_12032\4066791945.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  hourly_data = testing.resample(f'{resample_len}H').mean()


In [97]:
hourly_data = hourly_data.round(1)
hourly_data

,Datetime,Hour,Temperature,Dew Point,Humidity,Wind,Speed,Gust,Pressure,Precip.Rate.,Precip.Accum.,UV,Solar,Date
0,2023-01-01 00:00:00,1,13.7,12.8,93.2,258.8,0.0,NaN,29.9,0.0,0.0,0.0,0.0,2023-01-01
1,2023-01-01 01:00:00,2,13.5,12.8,94.0,249.8,0.0,NaN,29.9,0.0,0.0,0.0,0.0,2023-01-01
2,2023-01-01 02:00:00,3,13.5,12.8,94.0,279.0,0.0,NaN,29.9,0.0,0.0,0.0,0.0,2023-01-01
3,2023-01-01 03:00:00,4,13.1,12.1,94.0,290.2,0.0,NaN,29.9,0.0,0.0,0.0,0.0,2023-01-01
4,2023-01-01 04:00:00,5,12.4,11.7,94.0,234.0,0.0,NaN,29.9,0.0,0.0,0.0,0.0,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00,20,10.4,1.2,53.9,229.5,0.0,NaN,30.0,0.0,0.0,0.0,0.0,2023-12-31
8756,2023-12-31 20:00:00,21,9.9,1.1,55.1,235.0,0.0,NaN,30.0,0.0,0.0,0.0,0.0,2023-12-31
8757,2023-12-31 21:00:00,22,9.4,1.1,56.1,229.5,0.0,NaN,30.0,0.0,0.0,0.0,0.0,2023-12-31
8758,2023-12-31 22:00:00,23,8.9,0.9,57.9,231.8,0.0,NaN,30.0,0.0,0.0,0.0,0.0,2023-12-31


In [98]:
a = epw()
a.read('USA_GA_Atlanta-Hartsfield-Jackson.Intl.AP.722190_TMY3.epw')
test_df = a.dataframe
comparison = a.dataframe

In [99]:
# Define the custom conversion function
def convert_inhg_to_pa(series):
    return series * 3386.39

In [100]:
test_df['Year'] = 2023
test_df['Dry Bulb Temperature'] = hourly_data['Temperature']
test_df['Dew Point Temperature'] = hourly_data['Dew Point']
test_df['Relative Humidity'] = hourly_data['Humidity']#.astype(int)
test_df['Wind Direction'] = hourly_data['Wind']
test_df['Wind Speed'] = hourly_data['Speed']
# Convert the 'Pressure_inHg' column to Pascals
test_df['Atmospheric Station Pressure'] = hourly_data['Pressure'].apply(convert_inhg_to_pa)

In [106]:
id = 14

print(test_df.columns[id])
print(test_df.iloc[:,id])

Direct Normal Radiation
0       0
1       0
2       0
3       0
4       0
       ..
8755    0
8756    0
8757    0
8758    0
8759    0
Name: Direct Normal Radiation, Length: 8760, dtype: int64


In [54]:
test_df

,Year,Month,Day,Hour,Minute,Data Source and Uncertainty Flags,Dry Bulb Temperature,Dew Point Temperature,Relative Humidity,Atmospheric Station Pressure,...,Ceiling Height,Present Weather Observation,Present Weather Codes,Precipitable Water,Aerosol Optical Depth,Snow Depth,Days Since Last Snowfall,Albedo,Liquid Precipitation Depth,Liquid Precipitation Quantity
0,2023,1,1,1,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,13.7,12.8,93,0.8,...,910,9,999999999,170,0.052,0,88,999.0,999.0,99.0
1,2023,1,1,2,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,13.5,12.8,94,0.8,...,910,9,999999999,170,0.052,0,88,999.0,999.0,99.0
2,2023,1,1,3,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,13.5,12.8,94,0.8,...,850,9,999999999,170,0.052,0,88,999.0,999.0,99.0
3,2023,1,1,4,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,13.1,12.1,94,0.8,...,1160,9,999999999,179,0.052,0,88,999.0,999.0,99.0
4,2023,1,1,5,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,12.4,11.7,94,0.8,...,1010,9,999999999,179,0.052,0,88,999.0,999.0,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023,12,31,20,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,10.4,1.2,53,0.8,...,240,9,999999999,129,0.053,0,88,999.0,999.0,99.0
8756,2023,12,31,21,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,9.9,1.1,55,0.8,...,270,9,999999999,129,0.053,0,88,999.0,999.0,99.0
8757,2023,12,31,22,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,9.4,1.1,56,0.8,...,340,9,999999999,129,0.053,0,88,999.0,999.0,99.0
8758,2023,12,31,23,0,?9?9?9?9E0?9?9?9?9?9?9?9?9?9?9?9?9?9?9?9*9*9*9...,8.9,0.9,57,0.8,...,340,9,999999999,129,0.053,0,88,999.0,999.0,99.0


In [108]:
a.write('Final_CRC.epw')

In [109]:
from uwg import UWG

# Define the .epw, .uwg paths to create an uwg object.
epw_path = "Final_CRC.epw" # available in resources directory.
#USA_GA_Fulton.County.AP-Brown.Field.722195_TMY3.epw

# Initialize the UWG model by passing parameters as arguments, or relying on defaults
model = UWG.from_param_args(epw_path=epw_path, bldheight=11, blddensity=0.1, 
                                vertohor=0.8, charlength=125, sensanth=0.835,
                                treecover=0.2, grasscover=0.6, vegstart=3, 
                                vegend=11, albveg=0.15, h_obs= 3, 
                                h_ubl1=700, windmin=3,
                                zone='7', nday=30)

model.generate()
model.simulate()

# Write the simulation result to a file.
model.write_epw()


Simulating new temperature and humidity values for 30 days from 1/1.
Simulating Day 1
Simulating Day 2
Simulating Day 3
Simulating Day 4
Simulating Day 5
Simulating Day 6
Simulating Day 7
Simulating Day 8
Simulating Day 9
Simulating Day 10
Simulating Day 11
Simulating Day 12
Simulating Day 13
Simulating Day 14
Simulating Day 15
Simulating Day 16
Simulating Day 17
Simulating Day 18
Simulating Day 19
Simulating Day 20
Simulating Day 21
Simulating Day 22
Simulating Day 23
Simulating Day 24
Simulating Day 25
Simulating Day 26
Simulating Day 27
Simulating Day 28
Simulating Day 29
Simulating Day 30
New climate file is generated at Final_CRC_UWG.epw.
